In [4]:
# dependencies 
import pandas as pd
import random as rd
import numpy as np
import matplotlib.pyplot as plt
import timeit
import time
import requests
import gmaps
import json
#from pandas.io.json import json_normalize 

# libraries and modules
from datetime import datetime
from citipy import citipy
from scipy.stats import linregress
from bs4 import BeautifulSoup

# api keys
from config import weather_api_key
from config import g_key

In [5]:
# create a new set of 2,000 random latitudes and longitudes.

lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# zip into tuples and unzip into list
lat_lngs = zip(lats, lngs)
coordinates = list(lat_lngs)

# create list of cities from coordinates
cities = []
for c in coordinates:
    city = citipy.nearest_city(c[0], c[1]).city_name

    if city not in cities:
        cities.append(city)
len(cities)

# perform an api call with Open Weather Map and retrieve
# URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list. -- LEARN ENUMERATE!
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# lat long max temp humidity cloudiness windspeed weather descripton

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()

        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]

        # always note if its a list or dictionary
        weather_desc = city_weather["weather"][0]["description"]

        # Convert the date to ISO standard.
        #city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Weather Description": weather_desc})
                          #"Date": city_date})

# If an error is experienced, skip the city.
    except:
        
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

#Re-order 
new_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Weather Description"]
city_data_df = city_data_df[new_order]
city_data_df.head(30)

# export as csv file
city_data_df.to_csv("WeatherPy_Database.csv")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | bluff
Processing Record 2 of Set 1 | vila franca do campo
Processing Record 3 of Set 1 | luanda
Processing Record 4 of Set 1 | yarega
Processing Record 5 of Set 1 | sentyabrskiy
City not found. Skipping...
Processing Record 6 of Set 1 | santa lucia
Processing Record 7 of Set 1 | busselton
Processing Record 8 of Set 1 | ushuaia
Processing Record 9 of Set 1 | sault sainte marie
Processing Record 10 of Set 1 | bethel
Processing Record 11 of Set 1 | avarua
Processing Record 12 of Set 1 | port elizabeth
Processing Record 13 of Set 1 | hai duong
Processing Record 14 of Set 1 | qaanaaq
Processing Record 15 of Set 1 | jiuquan
Processing Record 16 of Set 1 | beloha
Processing Record 17 of Set 1 | los llanos de aridane
Processing Record 18 of Set 1 | lavrentiya
Processing Record 19 of Set 1 | valparaiso
Processing Record 20 of Set 1 | fairbanks
Processing Record 21 of Set 1 | khatanga
Processing Record 22 